In [ ]:
#Kayden Scheer
#Joao Franchi
#Henry Torres

In [2]:
sensor_readings = [22.5, 22.7, 22.6, 22.9, 23.1, 23.4, 23.8, 24.0, 24.3, 24.8]

for value in sensor_readings:
    print(value)

#No control
#Data automatically flows

22.5
22.7
22.6
22.9
23.1
23.4
23.8
24.0
24.3
24.8


In [4]:
it = iter(sensor_readings)

next(it)
next(it)
next(it)

#Because it's an iterator, data only flows when next() is called
#Iterator remembers its place so next call will be the next value

22.6

In [7]:
def sensor_stream(data):
    for value in data:
        print("Producing:", value)
        yield value

stream = sensor_stream(sensor_readings)

next(stream)
next(stream)
next(stream)

#yield pauses the function, doesn't destroy it
#remembers its place like next()

Producing: 22.5
Producing: 22.7
Producing: 22.6


22.6

In [22]:
def normal_func():
    print("Start")
    return 1
    print("End")

normal_func()

#return destroys function

Start


1

In [29]:
def gen_func():
    print("Start")
    yield 1
    print("Middle")
    yield 2
    print("End")

g = gen_func()

next(g)
next(g)
next(g)

#error is thrown, not entirley sure why, probably has something to do with yield saving teh function state

Start
Middle
End


StopIteration: 

In [15]:
def counter():
    x = 0
    while True:
        yield x
        x+=1

c = counter()
next(c)
next(c)
next(c)

#doesn't reset x

2

In [31]:
def high_temp_filter(stream, threshold):
    for value in stream:
        print("Filtering:", value)
        if value > threshold:
            yield value

stream = sensor_stream(sensor_readings)
filtered = high_temp_filter(stream, 23.0)

next(filtered)
next(filtered)

#next() and yield together allows for precise control
#data will keep flowing from one next() call until yield succeeds the check

Producing: 22.5
Filtering: 22.5
Producing: 22.7
Filtering: 22.7
Producing: 22.6
Filtering: 22.6
Producing: 22.9
Filtering: 22.9
Producing: 23.1
Filtering: 23.1
Producing: 23.4
Filtering: 23.4


23.4

In [21]:
class User:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __str__(self):
        return f"{self.name}, age {self.age}"

    def __repr__(self):
        return f"User(name={self.name!r}, age={self.age})"

u = User("Alice", 21)
print(u)
u

Alice, age 21


User(name='Alice', age=21)

In [24]:
class Cart:
    def __init__(self, items):
        self.items = items

    def __len__(self):
        return len(self.items)

    def __bool__(self):
        return len(self.items) > 0

cart = Cart(["apple", "banana"])
empty_cart = Cart([])

len(cart)
bool(cart)

if cart:
    print("Cart has items")

Cart has items


In [25]:
class Log:
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
        
log = Log([10, 20, 30, 40])
log[0]
log[2]
for x in log:
    print(x)

10
20
30
40


In [26]:
class Score:
    def __init__(self, value):
        self.value = value
    
    def __eq__(self, other):
        return self.value == other.value
    
    def __lt__(self, other):
        return self.value < other.value
    
    def __add__(self, other):
        return Score(self.value + other.value)

a = Score(10)
b = Score(20)
a == b
a < b
c = a + b
c.value

30

In [28]:
gen = sensor_stream(sensor_readings)

iter(gen) is gen
next(gen)

#all methods with __name__ formatting are dunder methods
#they happen automatically in Python's background

Producing: 22.5


22.5

In [4]:
def outer():
    x=10
    def inner():
        print("x =", x)
    return inner

f = outer()
f()
f()

#x is stored in the heap because its stored in teh function object
#x still exists because it's in the heap, which continues to exist after the stack frame is destroyed
#execution does reset because the stack frame is destoryed, but Python remembers the data stored in x

x = 10
x = 10


In [5]:
def counter():
    count = 0
    def inc():
        nonlocal count
        count +=1
        print("count =", count)
    return inc

c = counter()
c()
c()
c()

#count is being preserved in heap, so even after the new stack is created, Python remembers the value
#it still lives in heap

count = 1
count = 2
count = 3


In [7]:
print(c.__closure__)
print(c.__closure__[0].cell_contents)

(<cell at 0x000001AC1A2658D0: int object at 0x00007FFAC5A9B3E8>,)
3


In [9]:
def my_decorator(func):
    def wrapper():
        print("Before function")
        func()
        print("After function")
    return wrapper

@my_decorator
def greet():
    print("Hello!")

greet()

#my_decorator runs first
#no, we did not modify greet(), we just added the decorator features onto it
#my_decorator added the extra behavior

Before function
Hello!
After function


In [12]:
def demo_decorator(func):
    print("DECORATOR runs (definition time)")
    def wrapper():
        print("WRAPPER start")
        func()
        print("WRAPPER end")
    return wrapper

@demo_decorator
def say_hi():
    print("HI")

say_hi()

#when the function is defined as a decorator, it runs, showing that the wrapper doesn't run immediatly when the decorator is called
#the wrapper is modification of the function, so the function runs the decorator every time its called
#the modifications often change how the function runs in some way, which means the wrapper should call the original function when it is needed

DECORATOR runs (definition time)
WRAPPER start
HI
WRAPPER end


In [13]:
print(say_hi)
print(say_hi.__closure__)

#the wrapper overrides the original function, so it now refers to the wrapper class
#the original function is there so the wrapper can access the original functionality

<function demo_decorator.<locals>.wrapper at 0x000001AC1A24E5C0>
(<cell at 0x000001AC1A2644C0: function object at 0x000001AC1A24FA60>,)


In [15]:
def log_calls(func):
    def wrapper(*args):
        print("Calling with args:", args)
        return func(*args)
    return wrapper

@log_calls
def add(a, b):
    return a + b

print(add(3, 4))
print(add(10, 20))

#logging is added once the original function is called
#no, the decorator only adds on to the function
#we don't know how many numbers we need to pass to the function so we use args

Calling with args: (3, 4)
7
Calling with args: (10, 20)
30


In [4]:
def make_multiplier(n):
    def multiply(x):
        return x*n
    return multiply

double = make_multiplier(2)
triple = make_multiplier(3)

print(double(5))
print(triple(5))

10
15


In [5]:
def loud(func):
    def wrapper():
        print ("LOUD MODE")
        func()
    return wrapper

@loud
def speak():
    print("hello")

speak()

LOUD MODE
hello
